In [8]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.cluster import KMeans
import scipy
from scipy.optimize import minimize
import networkx as nx

In [9]:
def show(img):
    cv2.imshow("title",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [10]:
class Gaussian:
    def __init__(self, mean=np.zeros((3,1)), sigma=np.eye(3)):
        self.mean = np.array(mean)
        self.sigma = np.array(sigma)

    def compute_probability(self, x):
        return scipy.stats.multivariate_normal.pdf(np.array(x),mean=self.mean,cov=self.sigma)

    def update_parameters(self, data):
        self.mean = np.mean(data, axis=0)
        self.sigma = np.cov(data, rowvar=0)
        
    def set_parameter(self,mean,sigma):
        self.mean = np.array(mean)
        self.sigma = np.array(sigma)

In [11]:
class GMM:
    def __init__(self, K):
        self.K = K
        self.gaussians = [Gaussian() for _ in range(self.K)]
        self.weights = np.array([1.0/K]*K)

    def initialize_gmm(self, X):
        clusterer = KMeans(n_clusters=self.K, max_iter=10, random_state=None)
        clusters = clusterer.fit_predict(X)

        num_pixels = float(X.shape[0])
        
        for i, gaussian in enumerate(self.gaussians):
            gaussian.update_parameters(X[clusters==i])
            self.weights[i] = np.sum(clusters==i)/num_pixels
            
        return clusters

    def get_component(self, x):
        components = np.zeros((x.shape[0], len(self.gaussians)))

        for i,g in enumerate(self.gaussians):
            components[:, i] = self.weights[i]*g.compute_probability(x)

        return np.argmax(components, axis=1)

    def compute_probability(self, x):
        return np.dot(self.weights, [g.compute_probability(x) for g in self.gaussians])
    
    def update_gmm(self, X, means, sigmas):
        num_pixels = X.shape[0]
        
        for i,gaussian in enumerate(self.gaussians):
            gaussian.set_parameter(means[i],sigmas[i])
            
        temp1 = np.zeros((num_pixels,5))
        temp2 = np.zeros((num_pixels))
        temp_weight = np.zeros(5)
        for i in range(5):
            temp1[:,i] = FG_GMM.gaussians[i].compute_probability(X)
        for i in range(num_pixels):
            temp2[i] = np.argmax(temp1[i])
        for i in range(5):
            temp_weight[i] = np.sum(temp2==i) / num_pixels

        self.weights = temp_weight

In [12]:
img = cv2.imread("image1.jpg")

xmin, ymin, xmax, ymax = 130,40,500,440
height, width, _ = img.shape
alpha = np.zeros((height, width), dtype=np.int8)

for h in range(height):
    for w in range(width):
        if (w >= xmin) and (w <= xmax) and (h >= ymin) and (h <= ymax):
            alpha[h,w] = 1

In [13]:
img12312 = cv2.imread("image1.jpg",0)
mask = np.full_like(img12312,1.0,dtype=np.float32)
mask[ymin:ymax,xmin:xmax] = 0

show(mask)

In [14]:
fore_ground = img[alpha == 1]
back_ground = img[alpha == 0]

FG_GMM = GMM(5)
BG_GMM = GMM(5)

In [15]:
FG_cluster = FG_GMM.initialize_gmm(fore_ground)
BG_cluster = BG_GMM.initialize_gmm(back_ground)

In [16]:
pixels = img.reshape((img.shape[0]*img.shape[1], img.shape[2]))

In [17]:
f_component_map = (FG_GMM.get_component(pixels).reshape((img.shape[0],img.shape[1])) ) *0.25
b_component_map = (BG_GMM.get_component(pixels).reshape((img.shape[0],img.shape[1])) ) *0.25

In [18]:
h1, w1 = f_component_map.shape[:2]
h2, w2 = b_component_map.shape[:2]
vis = np.zeros((max(h1, h2), w1+w2), np.float32)
vis[:h1, :w1] = f_component_map
vis[:h2, w1:w1+w2] = b_component_map
vis = cv2.cvtColor(vis, cv2.COLOR_GRAY2BGR)

show(vis)

In [19]:
k = np.ones((img.shape[0],img.shape[1]), dtype=np.float32)*-1
k[alpha==1] = f_component_map[alpha==1]
k[alpha==0] = b_component_map[alpha==0]
show(k)

In [16]:
import numpy as np
from scipy.optimize import minimize
from numdifftools import Jacobian, Hessian

def fun(x, a):
    return (x[0] - 1) **2 + (x[1] - a) **2

def fun_der(x, a):
    return Jacobian(lambda x: fun(x, a))(x).ravel()

x0 = np.array([2, 0]) # initial guess
a = 2.5

res = minimize(fun, x0, args=(a,), method='BFGS', jac=fun_der)
print(res)

      fun: 4.35845650134609e-29
 hess_inv: array([[0.93103448, 0.17241379],
       [0.17241379, 0.56896552]])
      jac: array([1.24344979e-14, 4.44088348e-15])
  message: 'Optimization terminated successfully.'
     nfev: 3
      nit: 2
     njev: 3
   status: 0
  success: True
        x: array([1. , 2.5])
